In [31]:
# !pip install rapidfuzz


In [1]:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process

/opt/homebrew/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# read datasets
left_dataset = pd.read_csv("/Users/andychen/Desktop/Columbia_U/CU_2023_Spring/APAN5210/left_dataset.csv")
right_dataset = pd.read_csv("/Users/andychen/Desktop/Columbia_U/CU_2023_Spring/APAN5210/right_dataset.csv")

In [3]:
# check information of both datasets
print(left_dataset.info())
print(right_dataset.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98509 entries, 0 to 98508
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   business_id  98509 non-null  int64  
 1   name         98509 non-null  object 
 2   address      98509 non-null  object 
 3   city         98509 non-null  object 
 4   state        98509 non-null  object 
 5   zip_code     98509 non-null  object 
 6   size         98509 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 5.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94585 entries, 0 to 94584
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   entity_id    94585 non-null  int64  
 1   name         94585 non-null  object 
 2   address      91787 non-null  object 
 3   city         94585 non-null  object 
 4   state        94585 non-null  object 
 5   postal_code  94548 non-null  float64
 6 

In [4]:
#check missing values of left_dataset 
left_missing_values = left_dataset.isna().sum()
print("missing values in left_dataset：\n", left_missing_values)

#check missing values of right_dataset 
right_missing_values = right_dataset.isna().sum()
print("missing values in right_dataset：\n", right_missing_values)

missing values in left_dataset：
 business_id    0
name           0
address        0
city           0
state          0
zip_code       0
size           0
dtype: int64
missing values in right_dataset：
 entity_id         0
name              0
address        2798
city              0
state             0
postal_code      37
categories       62
dtype: int64


In [5]:
#drop na values of 'address' column in right dataset
right_dataset = right_dataset.dropna(subset=['address'])

In [6]:
# remane 'postal_code' in right_dataset to 'zip_code' and fill na with 0
right_dataset = right_dataset.rename(columns={"postal_code": "zip_code"})
right_dataset['zip_code'] = right_dataset['zip_code'].fillna(0).astype(int)
right_dataset["zip_code"] = right_dataset["zip_code"].astype(str).str.split(".").str[0].astype(int)

In [7]:
#process 'zip_code' in left_dataset and change its type to integer
left_dataset["zip_code"] = left_dataset["zip_code"].str.split("-").str[0]
left_dataset["zip_code"] = left_dataset["zip_code"].astype(int)

In [8]:
#cleaned the symbols and blank spaces in 'city' column, then make all letters in 'name' and 'city' lowercase
for col in ["city"]:
    left_dataset[col] = left_dataset[col].str.lower().str.replace(r'\W+', '', regex=True)
    right_dataset[col] = right_dataset[col].str.lower().str.replace(r'\W+', '', regex=True)

left_dataset["name"] = left_dataset["name"].str.lower()
right_dataset["name"] = right_dataset["name"].str.lower()

In [9]:
#sort values of right dataset in the order of state, city, and zip_code
right_dataset = right_dataset.sort_values(["state", "city","zip_code"])

In [11]:
#set index of right dataset to enhance the matching efficiency
right_indexed = right_dataset.set_index(["state", "city","zip_code"])

In [57]:
# set threshold
threshold = 80

results = []

for index_left, row_left in left_dataset.iterrows():
    #filter right_dataset subsets with the same state, city, and zip_code
    state_city_zip = (row_left["state"], row_left["city"], row_left["zip_code"])
    
    try:
        right_subset = right_indexed.loc[state_city_zip]
    except KeyError:
        continue

    if not right_subset.empty:
        #use process.extractOne to quickly find the best address match
        best_address_match_data = process.extractOne(row_left["address"], right_subset["address"], scorer=fuzz.token_sort_ratio)
        best_address_match = best_address_match_data[0]
        best_address_score = best_address_match_data[1]
        
        
        
        #use process.extractOne to quickly find the best name match
        best_name_match_data = process.extractOne(row_left["name"], right_subset["name"], scorer=fuzz.token_sort_ratio)
        best_name_match = best_name_match_data[0]
        best_name_score = best_name_match_data[1]


        #calculate the combined score
        combined_score = (best_name_score + best_address_score) / 2

        #check if the match score is above the threshold
        if combined_score >= threshold:
            #Process the match here and add it to the result list
            match_id_left = row_left["business_id"]
            match_id_right = right_subset[right_subset["address"] == best_address_match].iloc[0]["entity_id"]
            results.append((match_id_left, match_id_right, combined_score))


In [ ]:
#transform the result into DataFrame
results_df = pd.DataFrame(results, columns=["left_business_id", "right_business_id", "confidence_score"])



#save the result in csv file
results_df.to_csv("E:/matched_businesses01.csv", index=False)

print("matched business and confidence score is saved in matched_businesses01.csv")